In [11]:
%load_ext autoreload
%autoreload 2
import duckdb
import os
import pandas as pd

import splink

from ibge import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
start = 1
limits = 0
url = "https://specieslink.net/ws/1.0/search?apikey=%s&offset=%d&limits=%d&kingdom=Plantae" % (os.environ["SPLINK"], start, limits)
print("start: %d limits: %d" % (start, limits))
print("url: %s")

start: 1 limits: 0
url: %s


In [ ]:
splink.download_specieslink()
print("total files: %d" % len(os.listdir("./parquet")))

start: 0 limits: 99 
start: 100 limits: 199 
start: 200 limits: 299 
start: 300 limits: 399 
start: 400 limits: 499 
start: 500 limits: 599 
start: 600 limits: 699 
start: 700 limits: 799 


In [46]:
# Initialize DuckDB connection
conn = duckdb.connect()

local_file = "./parquet"

conn.execute("DROP TABLE IF EXISTS specieslink")
conn.execute(f"""
    CREATE TABLE specieslink AS
    SELECT * FROM read_parquet('{local_file}', union_by_name=True)
""")

result = conn.execute("SELECT COUNT(*) FROM specieslink").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 22,000


In [47]:
conn.execute("DESCRIBE specieslink").df()

,column_name,column_type,null,key,default,extra
0,barcode,VARCHAR,YES,None,None,None
1,collectioncode,VARCHAR,YES,None,None,None
2,scientificname,VARCHAR,YES,None,None,None
3,kingdom,VARCHAR,YES,None,None,None
4,family,VARCHAR,YES,None,None,None
5,genus,VARCHAR,YES,None,None,None
6,yearcollected,VARCHAR,YES,None,None,None
7,monthcollected,VARCHAR,YES,None,None,None
8,daycollected,VARCHAR,YES,None,None,None
9,country,VARCHAR,YES,None,None,None


In [48]:
conn.execute("SELECT * FROM specieslink LIMIT 5").df()

,barcode,collectioncode,scientificname,kingdom,family,genus,yearcollected,monthcollected,daycollected,country,...,dayidentified,infraspecificepithet,typestatus,relationshiptype,relatedcatalogitem,coordinateprecision,phylum,class,order,fieldnumber
0,CONV005839,CONVOLVULACEAE_BR,Bonamia ferruginea,Plantae,Convolvulaceae,Bonamia,1974,07,29,Brazil,...,None,None,None,None,None,None,None,None,None,None
1,None,CPAP,None,Plantae,Poaceae,None,1989,09,23,Brasil,...,None,None,None,None,None,None,None,None,None,None
2,CONV005840,CONVOLVULACEAE_BR,Bonamia ferruginea,Plantae,Convolvulaceae,Bonamia,1974,07,29,Brazil,...,None,None,None,None,None,None,None,None,None,None
3,None,CPAP,None,Plantae,Poaceae,None,1991,04,11,Brasil,...,None,None,None,None,None,None,None,None,None,None
4,CONV005829,CONVOLVULACEAE_BR,Bonamia ferruginea,Plantae,Convolvulaceae,Bonamia,1975,08,04,Brazil,...,None,None,None,None,None,None,None,None,None,None


In [49]:
response = get_data_ibge()
data = [{"id": r["id"], "nome_cidade": r["nome"], "sigla": get_sigla(r), "nome_estado": get_nome_uf(r)} for r in response.json()]

df = pd.DataFrame.from_records(data)
conn.execute("DROP TABLE IF EXISTS ibge")
conn.execute(f"""
    CREATE TABLE ibge AS
    SELECT * FROM df
""")

result = conn.execute("SELECT COUNT(*) FROM ibge").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 5,571


In [50]:
conn.execute("SELECT * FROM ibge LIMIT 5").df()

,id,nome_cidade,sigla,nome_estado
0,1100015,Alta Floresta D'Oeste,RO,Rondônia
1,1100023,Ariquemes,RO,Rondônia
2,1100031,Cabixi,RO,Rondônia
3,1100049,Cacoal,RO,Rondônia
4,1100056,Cerejeiras,RO,Rondônia


In [51]:
conn.execute("SELECT specieslink.stateprovince FROM specieslink LIMIT 5").df()

,stateprovince
0,Amazonas
1,MS
2,Amazonas
3,MS
4,Amazonas


In [55]:
conn.execute("DROP TABLE IF EXISTS specieslink_sigla")
conn.execute("""
            CREATE TABLE specieslink_sigla AS
            SELECT
                COALESCE(ibge.sigla, specieslink.stateprovince) AS state_province
            FROM specieslink
                     LEFT JOIN ibge
                               ON specieslink.stateprovince = ibge.sigla
            """)

conn.execute("COPY specieslink_sigla TO 'specieslink_sigla.parquet' (FORMAT PARQUET)")

In [56]:
conn.execute("SELECT distinct(specieslink_sigla.state_province) FROM specieslink_sigla LIMIT 5").df()

,state_province
0,Pará
1,Paraná
2,Pastaza
3,Puno
4,Monagas


In [58]:
conn.execute("DROP TABLE IF EXISTS specieslink_nome_estado")
conn.execute("""
             CREATE TABLE specieslink_nome_estado AS
             SELECT
                 COALESCE(ibge.nome_estado, specieslink_sigla.state_province) AS state_province
             FROM specieslink_sigla
                      LEFT JOIN ibge
                                ON specieslink_sigla.state_province = ibge.nome_estado
             """)

conn.execute("COPY specieslink_nome_estado TO 'specieslink_nome_estado.parquet' (FORMAT PARQUET)")

In [60]:
conn.execute("SELECT distinct(specieslink_nome_estado.state_province) FROM specieslink_nome_estado LIMIT 5").df()

,state_province
0,Huánuco
1,Napo
2,Pichincha
3,Apurímac
4,Biobío


In [64]:
conn.execute("SELECT \
stateprovince, \
COUNT(DISTINCT family) AS num_families \
FROM specieslink \
WHERE family IS NOT NULL AND stateprovince IS NOT NULL \
GROUP BY stateprovince \
ORDER BY num_families DESC;").df()

,stateprovince,num_families
0,São Paulo,80
1,MS,64
2,Mato Grosso,62
3,Goiás,39
4,Amazonas,38
...,...,...
229,Rovinj,1
230,Edo Falcón,1
231,Zamora Chinchipe,1
232,Itapúa,1
